In [100]:
import os
import pandas as pd
import openpyxl, uuid, datetime, getpass, json

# Assumptions
#
# if unable to tell, use the first sheet


In [110]:
# Read files in the input directory for Excel items without accompanying .json document

wlk = {};
for r,d,f in os.walk('input'):
    for file in f:
        if file.endswith(".xlsx") and not file.startswith('~'):
            wlk[file] = True;
      
    for file in f:
        if file.endswith(".json") and os.path.splitext(file)[0] + '.xlsx' in wlk:
            wlk[os.path.splitext(file)[0] + '.xlsx'] = False;
 
rez = {};
for k,v in wlk.items():
    if v is True:
        rez[k] = {};
        
print("Found " + str(len(rez)) + " Excel files to process.")

Found 1 Excel files to process.


In [111]:
# Determine whether UST or LUST

for k,v in rez.items():
    print("Examining " + k);
    rez[k]['key'] = k;
    rez[k]['file'] = 'input' + os.sep + k;
    rez[k]['date_examined'] = datetime.datetime.today().strftime('%Y%m%d');
    rez[k]['username'] = getpass.getuser();
    certainty = 0;
    
    lowname = k.lower();
    if lowname.find('_lust.') > 0 or lowname.find('_lust_') > 0:
        rez[k]['datatype'] = 'LUST';
        certainty = 100;
        
    elif lowname.find('_ust.') > 0 or lowname.find('_ust_') > 0:
        rez[k]['datatype'] = 'UST';
        certainty = 100;
        
    elif lowname.find('_lust') > 0:
        rez[k]['datatype'] = 'LUST';
        certainty = 80;
        
    elif lowname.find('_ust') > 0:
        rez[k]['datatype'] = 'UST';
        certainty = 80;
        
    elif lowname.find('lust') > 0:
        rez[k]['datatype'] = 'LUST';
        certainty = 40;
        
    else:
        print("  unable to tell datatype from filename.");
        certainty = 0;
        
    if certainty < 100:
        xlsx = pd.ExcelFile('input' + os.sep + k);
        
        print("  checking sheet names for clues");
        for sht in xlsx.sheet_names:
            lowname = sht.lower();
            
            if lowname.find('_lust') > 0:
                rez[k]['datatype'] = 'LUST';
                certainty = 100;
                break;
            
            elif lowname.find(' lust') > 0:
                rez[k]['datatype'] = 'LUST';
                certainty = 100;
                break;
                
            elif lowname.find('_ust') > 0:
                rez[k]['datatype'] = 'UST';
                certainty = 100;
                break;
                
            elif lowname.find(' ust') > 0:
                rez[k]['datatype'] = 'UST';
                certainty = 100;
                break;
                
            if 'datatype' in rez[k]:
                if rez[k]['datatype'] == 'LUST' and lowname.find('lust') > 0:
                    rez[k]['datatype'] = 'LUST';
                    certainty = 100;
                    break;

                elif rez[k]['datatype'] == 'UST' and lowname.find('ust') > 0:
                    rez[k]['datatype'] = 'UST';
                    certainty = 100;
                    break;
                    
        del xlsx
    
    if certainty == 0:    
        rez[k]['error'] = 'Unable to determine UST or LUST';
        
    if 'error' in rez[k]:
        print("  " + rez[k]['error']);
    else:
        print("  determined datatype as " + rez[k]['datatype'] + ".");


Examining OR_LUST.xlsx
  determined datatype as LUST.


In [112]:
# Determine the sheet to process

for k,v in rez.items():
    if 'error' not in v:
        
        print("Examining " + k);
        xlsx = pd.ExcelFile('input' + os.sep + k);
        sht_count = len(xlsx.sheet_names)
        print("File has " + str(sht_count) + " sheets");
        
        if sht_count == 1:
            rez[k]['sheet_no']    = 0;
            rez[k]['sheet_name'] = xlsx.sheet_names[0];
                
        else:
            idx = 0;
            for sht in xlsx.sheet_names:
                lowname = sht.lower();

                if rez[k]['datatype'] == 'LUST' and lowname.find(' lust') > 0:
                    rez[k]['sheet_no']   = idx;
                    rez[k]['sheet_name'] = sht;
                    break;

                elif rez[k]['datatype'] == 'LUST' and lowname.find('_lust') > 0:
                    rez[k]['sheet_no']   = idx;
                    rez[k]['sheet_name'] = sht;
                    break;

                elif rez[k]['datatype'] == 'UST' and lowname.find('_ust') > 0:
                    rez[k]['sheet_no']   = idx;
                    rez[k]['sheet_name'] = sht;
                    break;

                elif rez[k]['datatype'] == 'UST' and lowname.find(' ust') > 0:
                    rez[k]['sheet_no']   = idx;
                    rez[k]['sheet_name'] = sht;
                    break;

                else:
                    rez[k]['error'] = 'Unable to determine sheet to process';
                
        del xlsx  
        
    if 'error' in rez[k]:
        print("  " + rez[k]['error']);
    else:
        print("  will process " + rez[k]['sheet_name'] + ".");
          

Examining OR_LUST.xlsx
File has 7 sheets
  will process OR_LUST.


In [113]:
# Determine fields to geocode

for k,v in rez.items():
    if 'error' not in v:
        
        print("Examining " + rez[k]['datatype'] + ' file ' + k + ', sheet ' + str(rez[k]['sheet_no']));
        df = pd.read_excel('input' + os.sep + k,sheet_name=rez[k]['sheet_no']);
        tr = len(df.index);
        print("Sheet has " + str(tr) + " records.");
        
        rez[k]['record_count'] = tr;
        rez[k]['mapping'] = {};
        rez[k]['has_globalid'] = False;
        mp = rez[k]['mapping'];
        
        for col in df.columns:
            lowname = col.lower();
            
            if lowname == 'globalid':
                rez[k]['has_globalid'] = True;
            
            # make sure we only take the first matching instance from left to right
            elif rez[k]['datatype'] == 'UST':
                if 'sourceidentifier' not in mp and lowname == 'facilityid':
                    mp['sourceidentifier'] = col;
             
                elif 'name' not in mp and lowname == 'facilityname':
                    mp['name'] = col;
                
                elif 'address1' not in mp and lowname in ['facilityaddress','facilityaddress1']:
                    mp['address1'] = col;
                    
                elif 'address2' not in mp and lowname == 'facilityaddress2':
                    mp['address2'] = col;
            
                elif 'address3' not in mp and lowname == 'facilityaddress3':
                    mp['address3'] = col;
                    
                elif 'city' not in mp and lowname == 'facilitycity':
                    mp['city'] = col;
                    
                elif 'county' not in mp and lowname == 'facilitycounty':
                    mp['county'] = col;
                    
                elif 'state' not in mp and lowname == 'facilitystate':
                    mp['state'] = col;
                    
                elif 'zip5' not in mp and lowname == 'zipcode':
                    mp['zip5'] = col;
                    
                elif 'zip4' not in mp and lowname == 'zip4':
                    mp['zip4'] = col;
                    
            elif rez[k]['datatype'] == 'LUST':
                if 'sourceidentifier' not in mp and lowname == 'facilityid':
                    mp['sourceidentifier'] = col;
             
                elif 'name' not in mp and lowname == 'sitename':
                    mp['name'] = col;
                
                elif 'address1' not in mp and lowname in ['siteaddress','siteaddress1']:
                    mp['address1'] = col;
                    
                elif 'address2' not in mp and lowname == 'siteaddress2':
                    mp['address2'] = col;
            
                elif 'address3' not in mp and lowname == 'siteaddress3':
                    mp['address3'] = col;
                    
                elif 'city' not in mp and lowname == 'sitecity':
                    mp['city'] = col;
                    
                elif 'county' not in mp and lowname in ['sitecounty','county']:
                    mp['county'] = col;
                    
                elif 'state' not in mp and lowname in ['sitestate','state']:
                    mp['state'] = col;
                    
                elif 'zip5' not in mp and lowname == 'zipcode':
                    mp['zip5'] = col;
                
                elif 'zip4' not in mp and lowname == 'zip4':
                    mp['zip4'] = col;
       
        del df;
        

Examining LUST file OR_LUST.xlsx, sheet 0
Sheet has 7317 records.


In [114]:
# Add globalids if not present
for k,v in rez.items():
    if 'error' not in v:
        
        if not v['has_globalid']:
            print("File " + k + ", sheet " + str(v['sheet_no']) + " lacks globalids.");
            
            wb = openpyxl.load_workbook('input' + os.sep + k);
            sheets = wb.sheetnames;

            ws = wb[sheets[v['sheet_no']]];
            
            print("Adding globalids (may take a while)...");
            newcol = ws.max_column + 1;
            ws.cell(
                 row    = 1
                ,column = newcol
                ,value  = 'globalid'
            );
            for row_num in range(2,ws.max_row):
                ws.cell(
                     row    = row_num
                    ,column = newcol
                    ,value  = '{' + str(uuid.uuid4()) + '}'
                );
            
            print("Saving workbook.");
            wb.save('input' + os.sep + k);
            del ws,wb;
            
        else:
            print("File " + k + ", sheet " + str(v['sheet_no']) + " has preexisting globalids.");
            

File OR_LUST.xlsx, sheet 0 has preexisting globalids.


In [115]:
# Write out the companion json file

for k,v in rez.items():
    if 'error' not in v:
        
        fn = 'input' + os.sep + os.path.splitext(k)[0] + '.json';
        if os.path.exists(fn):
            os.remove(fn);
        
        print("writing out companion file for " + k);
        with open(fn, 'w') as fp:
            json.dump(v,fp,indent = 3);
        

writing out companion file for OR_LUST.xlsx
